In [11]:
import os
import torch 
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR
from collections import Counter
from torchsummary import summary
from tokenizers import ByteLevelBPETokenizer
from transformers import AutoTokenizer
import json
import math

torch.manual_seed(22)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#pip install tokenizers transformers #Instalar en caso de ser necesario

## Cargar los archivos necesarios

In [37]:
file_path = 'all_poems.txt'
output_dir = "tokenizer_spanish_" ##Crear carpeta para guardar el entrenamiento del Tokenizador
# Crea el directorio si no existe
#os.makedirs(output_dir, exist_ok=True)

In [23]:
# Cargar el tokenizador existente
tokenizer = AutoTokenizer.from_pretrained("VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis")

In [39]:
with open('all_poems.txt', 'r', encoding='utf-8') as f:
  text = f.readlines()

## **Entrenar un tokenizador por subpalabras**

In [40]:
# Entrenar un nuevo tokenizador a partir del corpus de entrenamiento
new_tokenizer = tokenizer.train_new_from_iterator(text, 5000)

In [43]:
# Guardar el tokenizador completo
new_tokenizer.save_pretrained(output_dir)

('tokenizer_spanish_\\tokenizer_config.json',
 'tokenizer_spanish_\\special_tokens_map.json',
 'tokenizer_spanish_\\vocab.txt',
 'tokenizer_spanish_\\added_tokens.json',
 'tokenizer_spanish_\\tokenizer.json')

In [44]:
vocab = new_tokenizer.get_vocab() 
print(vocab)
print(len(vocab))

{'##nas': 1064, 'esperando': 3627, '##uest': 377, 'respuesta': 4355, 'tambor': 4392, '##ento': 1069, 'confes': 4836, '##cua': 3643, '##zar': 785, 'unos': 1521, 'quieren': 3526, '##gantes': 3435, '##bida': 3849, '##ida': 601, 'fren': 2192, 'cantaba': 4585, 'niña': 1712, 'pueda': 2108, '##uto': 2689, 'ron': 1921, 'semejante': 3307, 'verdadero': 3735, 'ā': 94, '##cias': 783, '##mon': 1013, 'cemen': 4754, 'preci': 1345, 'encierra': 3533, 'aho': 2282, '##cones': 3268, 'exi': 4060, 'fiero': 3222, 'muros': 2541, 'piernas': 3087, 'sabía': 4335, 'reg': 1076, 'había': 1174, 'nosotros': 956, 'sir': 1838, '##tima': 2343, 'tocar': 2884, '##st': 284, 'ciega': 2703, '##rán': 1958, 'acos': 3330, 'cons': 809, 'fuga': 2118, 'saben': 2668, 'irre': 3493, 'bellos': 3698, 'presa': 2851, 'malo': 4704, 'cayó': 3291, '##dos': 255, 'agua': 660, 'turba': 2426, '##vida': 2512, 'pas': 362, 'piel': 1472, 'conten': 1846, 'hasta': 559, 'cuida': 1963, 'extien': 3955, 'lími': 2967, 'fiebre': 3519, '##ces': 326, 'memor'

In [45]:
vocab_size = len(vocab)  # Definir el tamaño del vocabulario
print(f"Tamaño del vocabulario: {vocab_size}")

Tamaño del vocabulario: 5000


In [46]:
# Definir funciones de codificación y decodificación
def encode(text):
    # Codificar sin agregar los tokens especiales como [CLS] y [SEP]
    return tokenizer.encode(text, add_special_tokens=False)

def decode(ids):
    # Eliminar los tokens especiales si están en la secuencia codificada
    ids = [id_ for id_ in ids if id_ not in [tokenizer.cls_token_id, tokenizer.sep_token_id]]
    return tokenizer.decode(ids)

## **Pruebas con base en el tokenizador entrenado**

In [47]:
# Texto de prueba
texto_prueba = "Este es un ejemplo de prueba para verificar el tokenizador. Hola mundo"

# Codificar el texto
encoded_text = encode(texto_prueba)
print("Texto codificado (IDs):", encoded_text)

# Decodificar el texto
decoded_text = decode(encoded_text)
print("Texto decodificado:", decoded_text)

Texto codificado (IDs): [1277, 1028, 1044, 2670, 1009, 3610, 1097, 14030, 1039, 10419, 1017, 14962, 1008, 1734, 1758]
Texto decodificado: este es un ejemplo de prueba para verificar el tokenizador. hola mundo


In [50]:
# Definir una porción del texto original para la prueba
sample_text = ''.join(text[:500])  # Toma los primeros 200 caracteres del texto original para la prueba

# Codificar la porción del texto
encoded_sample = encode(sample_text)
print("IDs codificados:", encoded_sample)

# Decodificar los IDs codificados
decoded_sample = decode(encoded_sample)
print("Texto decodificado:\n",decoded_sample)

Token indices sequence length is longer than the specified maximum sequence length for this model (3410 > 512). Running this sequence through the model will result in indexing errors


IDs codificados: [1040, 1493, 1019, 1041, 11296, 30958, 1040, 1054, 1094, 6614, 1019, 1048, 13888, 5507, 8978, 30958, 1035, 1039, 5571, 4215, 1019, 22531, 9532, 1203, 6951, 1035, 1032, 1771, 30956, 1009, 17768, 1019, 1041, 17049, 1151, 1091, 12426, 15004, 1040, 4307, 2037, 1548, 1019, 1120, 1155, 24450, 1021, 1019, 27302, 30958, 1040, 1155, 8250, 1012, 1714, 1109, 1008, 1008, 1203, 8193, 1157, 1214, 30063, 15287, 1009, 4132, 2182, 1039, 10125, 24766, 1492, 3, 1008, 11308, 1044, 4523, 1009, 7922, 1097, 13200, 1085, 5806, 14191, 1081, 1526, 1035, 1032, 2618, 1008, 1139, 1054, 1669, 6144, 995, 1054, 1669, 3264, 995, 1669, 1091, 1626, 1035, 7420, 5282, 1009, 1032, 1613, 1040, 1081, 1744, 1008, 1139, 3712, 995, 8767, 995, 1581, 1044, 1819, 1151, 1493, 1019, 1040, 1012, 2458, 2209, 1669, 2401, 1019, 1040, 1012, 2045, 2466, 8865, 1035, 1277, 10931, 1019, 1040, 1628, 1748, 1091, 3431, 4612, 1019, 9411, 1019, 1553, 6225, 1008, 1096, 1019, 1012, 1233, 5346, 1032, 5538, 10125, 1040, 14223, 13068,

## **Carga de datos**

In [52]:
# Codificar todo el texto y convertir a tensores de PyTorch
text = ''.join(text)
encoded_data = encode(text)
data = torch.tensor(encoded_data, dtype=torch.long)

In [53]:
#Dividir en entrenamiento y validación
n = int(0.85 * len(data))
train_data = data[:n]
val_data = data[n:]

In [54]:
len(train_data)

3012632

In [55]:
batch_size = 128
context_size = 128

# para generar los datos del modelo de lenguaje causal (predecir siguiente token)
# las entradas son porciones de texto codificadas de tamaño context_size y
# las salidas son las mismas porciones de texto recorridas un paso
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - context_size - 1, (batch_size,))
    x = data[ix.unsqueeze(1) + torch.arange(context_size)]  # (batch_size, context_size)
    y = data[ix.unsqueeze(1) + torch.arange(1, context_size + 1)]  # (batch_size, context_size)
    x, y = x.to(device), y.to(device)
    return x, y

In [56]:
get_batch("train")[0].shape

torch.Size([128, 128])

## **Hiperparámetros**

In [58]:
max_iters = 600_00      # pasos de entrenamiento
eval_interval = 1_000    # cada cuántos pasos calcular (train/valid) durante el entrenamiento
learning_rate = 1e-4
eval_iters = 200       # tamaño de la muestra para promediar en el cálculo de la pérdida
n_embd = 256             # tamaño de los embeddings internos
n_head = 8             # numero de cabezas de auto-atención
n_layer = 6            # numero de bloques Transformers
dropout = 0.3          # aplicado después de cada autoatención, FF, y enmascarado

In [59]:
class ProductoPuntoEscalado(nn.Module):
  def __init__(self,
               p_dropout = 0.0,
               masc = False):
    super(ProductoPuntoEscalado, self).__init__()
    self.masc = masc
    self.dropout = nn.Dropout(p_dropout)

  def forward(self, Q, K, V):
    # Obtenemos dimensiones
    m, n_cabezas, l, d_k = K.shape
    d_v = V.shape[-1]

    # Cambiamos la forma: [m, n_cabezas, l, d_k] -> [m * n_cabezas, l, d_k]
    Q = Q.reshape(m * n_cabezas, l, d_k)
    K = K.reshape(m * n_cabezas, l, d_k)
    V = V.reshape(m * n_cabezas, l, d_v)

    # Q y K tienen forma [m * n_cabezas, l, d_k],
    # por lo que se transponen las dos últimas dimensiones de K
    # QK: [m * n_cabezas, l, l]
    QK = torch.bmm(Q, K.transpose(1, 2))

    # se escalan los valores QK
    QK_esc = QK / torch.math.sqrt(d_k)

    if self.masc:
      # Creamos una matriz triangular superior binaria (excluyendo la diagonal)
      masc = torch.triu(torch.ones((l, l), dtype = torch.bool, device = Q.device),
                    diagonal = 1)
      # Ponemos los valores de QK_esc en los que la máscara sea 1 a -inf
      QK_esc = QK_esc.masked_fill_(masc, -torch.inf)

    # mapas de atención: [m * n_cabezas, l, l] -> [m * n_cabezas, l, l]
    alfas = nn.functional.softmax(QK_esc, dim=-1)
    alfas = self.dropout(alfas) # Se agrega dropout de acuerdo al codigo de nanoGPT

    # vectores de salida y
    # alfas: [m * n_cabezas, l, l], V: [m * n_cabezas, l, d_v]
    # Y: [m * n_cabezas, l, d_v]
    Y = torch.bmm(alfas, V)

    # Cambiamos la forma: [m * n_cabezas, l, d_v] -> [m, n_cabezas, l, d_v]
    Y = Y.reshape(m, n_cabezas, l, d_v)

    # Cambiamos la forma: [m * n_cabezas, l, l] -> [m, n_cabezas, l, l]
    alfas = alfas.reshape(m, n_cabezas, l, l)

    return Y, alfas


class AtencionMulticabeza(nn.Module):
  def __init__(self,
               d_modelo,
               n_cabezas,
               p_dropout = 0.0,
               masc = False):
    super(AtencionMulticabeza, self).__init__()

    self.n_cabezas = n_cabezas
    self.d_modelo = d_modelo

    self.d_cabezas = self.d_modelo // self.n_cabezas

    self.ppe = ProductoPuntoEscalado(p_dropout=p_dropout, masc = masc)
    self.proy_Q = nn.Linear(self.d_modelo, self.d_modelo, bias = False)
    self.proy_K = nn.Linear(self.d_modelo, self.d_modelo, bias = False)
    self.proy_V = nn.Linear(self.d_modelo, self.d_modelo, bias = False)
    self.proy_sal = nn.Linear(self.d_modelo, self.d_modelo)

  def forward(self, x):
    m, l, d_modelo = x.shape

    # Cambiamos la forma del tensor x
    # [m, l, d_modelo] -> [m * l, d_modelo]
    x = x.reshape(m * l, d_modelo)

    # Proyectamos vectores en x a Q, K, V
    # [m * l, d_modelo] -> [m * l, d_modelo]
    Q = self.proy_Q(x)
    K = self.proy_K(x)
    V = self.proy_V(x)

    # Cambiamos la forma: [m * l, d_modelo] -> [m, l, n_cabezas, d_k]
    # d_k = d_v = self.d_modelo // self.n_cabezas
    Q = Q.reshape(m, l, self.n_cabezas, self.d_cabezas)
    K = K.reshape(m, l, self.n_cabezas, self.d_cabezas)
    V = V.reshape(m, l, self.n_cabezas, self.d_cabezas)

    # Transponemos el eje de las cabezas a la segunda posición del tensor y
    # creamos copia (con .contiguous()) para que esté almacenado en memoria de
    # forma contigua (.transpose() hace que ya no sea así).
    # [m, l, n_cabezas, d_k] -> [m, n_cabezas, l, d_k]
    Q = Q.transpose(1, 2).contiguous()
    K = K.transpose(1, 2).contiguous()
    V = V.transpose(1, 2).contiguous()

    # Calculamos el producto punto escalado con Q, K y V
    # Q, K: [m, n_cabezas, l, d_k], V:[m, n_cabezas, l, d_v]
    # Y: [m, n_cabezas, l, d_v], alfas: [m, n_cabezas, l, l]
    Y, alfas = self.ppe(Q, K, V)

    # Transponermos el eje de cabezas a la penúltima posición:
    # [m, n_cabezas, l, d_k] -> [m, l, n_cabezas, d_k]
    Y = Y.transpose(1, 2).contiguous()

    # Concatemanos los vectores de todas las cabezas en un solo vector
    # [m, l, n_cabezas, d_k] -> [m * l, d_modelo]
    # d_modelo = n_cabezas * d_k
    Y = Y.reshape(m * l, self.d_modelo)

    # Proyectamos la vectores concatenados para obtener la salida
    # [m * l, d_modelo] -> [m * l, d_modelo]
    Y = self.proy_sal(Y)

    # Concatemanos los vectores de todas las cabezas en un solo vector
    # [m * l, d_modelo] -> [m, l, d_modelo]
    Y = Y.reshape(m, l, self.d_modelo)

    return Y, alfas

class RedDensaPosicion(nn.Module):
  def __init__(self,
               d_modelo,
               d_ff):
    super(RedDensaPosicion, self).__init__()
    self.d_modelo = d_modelo
    self.d_ff = self.d_ff = d_ff if d_ff else 4*d_modelo
    self.densa1 = nn.Linear(self.d_modelo, self.d_ff)
    self.densa2 = nn.Linear(self.d_ff, self.d_modelo)

  def forward(self, x):
    m, l, d_modelo = x.shape

    # Cambiamos la forma: [m, l, d_modelo] -> [m * l, d_modelo]
    x = x.reshape(m * l, d_modelo)

    # Pasamos el tensor redimensionado por la red densa
    # [m * l, d_modelo] -> [m * l, d_modelo]
    x = self.densa1(x)
    x = nn.functional.gelu(x)
    x = self.densa2(x)

    # Lo regresamos a su forma original
    # [m * l, d_modelo] -> [m, l, d_modelo]
    x = x.reshape(m, l, d_modelo)

    return x

class BloqueTransformer(nn.Module):
  def __init__(self,
               d_modelo,
               n_cabezas,
               d_rdp=None,
               p_dropout = 0.1,
              masc = False):
    super(BloqueTransformer, self).__init__()
    self.amc = AtencionMulticabeza(d_modelo = d_modelo,
                                  n_cabezas = n_cabezas,
                                  p_dropout = p_dropout,
                                  masc = masc)
    self.norm1 = nn.LayerNorm(d_modelo)
    self.rp = RedDensaPosicion(d_modelo, d_rdp)
    self.norm2 = nn.LayerNorm(d_modelo)
    self.dropout1 = nn.Dropout(p_dropout)
    self.dropout2 = nn.Dropout(p_dropout)

  def forward(self, x):
    salidas_amc, alfas = self.amc(x)
    salidas_amc = self.dropout1(salidas_amc)
    salidas_amc = self.norm1(x + salidas_amc)

    salidas_rp = self.rp(salidas_amc)
    salidas_rp = self.dropout2(salidas_rp)

    return self.norm2(salidas_amc + salidas_rp)


class CodificacionPosicional(nn.Module):
  def __init__(self,
               maxsec,
               d_modelo,
               p_dropout = 0.1):
    super(CodificacionPosicional, self).__init__()

    self.maxsec = maxsec
    self.d_modelo = d_modelo

    cod_pos = torch.zeros((self.maxsec, self.d_modelo))

    # Creamos tensor con valores pares 0, 2, 4, ...
    # i: [d_modelo // 2, 1]
    i = torch.arange(0, self.d_modelo, 2, dtype=torch.float).reshape(-1, 1)

    # Creamos tensor de posiciones 0, 1, ...
    # pos: [maxsec, 1]
    pos = torch.arange(0, self.maxsec, dtype=torch.float).reshape(-1, 1)
    a = 1.0 / 10000**(i / self.d_modelo)

    # grados: [maxsec, d_modelo // 2]
    grados = pos @ a.T

    cod_pos[:, 0::2] = torch.sin(grados) # Para pares
    cod_pos[:, 1::2] = torch.cos(grados) # Para impares

    # Registramos tensor de codificación posicional
    self.register_buffer('cod_pos', cod_pos)

    self.dropout = nn.Dropout(p_dropout)

  def forward(self, x):
    m, l, d_modelo = x.shape
    return x + self.cod_pos[:l, :]

In [60]:
class CodificacionPosicional(nn.Module):
    def __init__(self, context_size, n_embd):
        super().__init__()
        self.pe = torch.zeros(context_size, n_embd)
        position = torch.arange(0, context_size).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, n_embd, 2) * -(math.log(10000.0) / n_embd))
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = self.pe.unsqueeze(0)
    
    def forward(self, x):
        x = x + self.pe[:, :x.size(1)].to(x.device)
        return x


In [61]:
class nanoGPT(nn.Module):
    def __init__(self):
        super().__init__()
        # Tabla de embeddings para tokens
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        # Codificación posicional sinusoidal
        self.position_embedding = CodificacionPosicional(context_size, n_embd)
        # Dropout después de embeddings
        self.dropout = nn.Dropout(dropout)
        # Bloques Transformer
        self.blocks = nn.Sequential(*[BloqueTransformer(n_embd, n_cabezas=n_head, p_dropout=dropout, masc=True) for _ in range(n_layer)])
        # LayerNorm final
        self.ln_f = nn.LayerNorm(n_embd)
        # Capa de salida
        self.lm_head = nn.Linear(n_embd, vocab_size)
        # Compartir pesos entre embeddings y capa de salida
        self.lm_head.weight = self.token_embedding_table.weight
        # Inicialización de pesos
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0, std=0.02)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        x = self.position_embedding(tok_emb)  # (B,T,C)
        x = self.dropout(x)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)
        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets, label_smoothing=0.1)
        return logits, loss
    
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -context_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                values, indices = torch.topk(logits, top_k)
                logits[logits < values[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [62]:
model = nanoGPT()
m = model.to(device)

# imprime el número de parámetros en el modelo
print(sum(p.numel() for p in m.parameters()) / 1e6, 'M parameters')

6.019464 M parameters


In [63]:
# Durante el entrenamiento, cada (eval_interval) pasos, se obtienen
# (eval_iters) perdidas y se promedian para monitorear el estado del
# entrenamiento

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out

In [65]:
# Crear un contexto inicial vacío (en este caso un token de inicio en cero)
context = torch.zeros((1, 1), dtype=torch.long, device=device)

# Generar una secuencia de 2000 tokens desde el contexto inicial
generated_tokens = m.generate(context, max_new_tokens=2000)[0].tolist()

# Decodificar los tokens generados usando el tokenizador
# Suponiendo que 'tokenizer' es tu tokenizador configurado
generated_text = new_tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(generated_text)

##cada arranca música tiran arriba heregn rit santos luceros felici músi huracán lírioso frentetriz harábar muertodó honda rayo plazagas armon nebsovido all brillan exh males guarda cabo cari excla ruiseñ prome verdu algún apare lazos irre escondenidad vamosangzonridad grito costu sentado otraici silencios persellanula qui interros quemaur árbollicotera llano profundo asom pasos decir tranquilo î ign firmamentorp crea7 dormido perman cosaciente imagina ellamerainostro amb condeangineicos gemido hemos cu cen aban vacío bajo madu aquellaseri def estimá cordero cruzlgido semblantequí serpiente caballo ribera diste ofren lis tuyosrosaombre infer adora rue ellorocíamando mucha pétatura respon llevan au llena cierto dueletado sufri verdu cuerpo puro signi pequeña bocas quieren humoabas bas secatadotadas gracias puedes brin respira lágrimas naranam obra verte rubiacho héroorada sabes oricita subgidos segura tej hijo quieres gob púrpuracana algunas rol ren melodía playas adán niñotero op ej gu

In [66]:
# Crear optimizador y scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-3)
scheduler = CosineAnnealingLR(optimizer, T_max=max_iters)
scaler = GradScaler()

In [68]:
# Bucle de entrenamiento
for iter in range(max_iters):
    # Calcular la pérdida cada eval_interval
    if iter % eval_interval == 0 or iter == max_iters - 1:
        model.eval()
        with torch.no_grad():
            losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        model.train()

    # Generar lote
    xb, yb = get_batch('train')

    # Paso de entrenamiento con AMP
    optimizer.zero_grad()
    with autocast():
        logits, loss = model(xb, yb)
    scaler.scale(loss).backward()
    scaler.unscale_(optimizer)
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    scaler.step(optimizer)
    scaler.update()
    scheduler.step()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [139]:
model_save_path = 'nanoGPT_model.pth'
torch.save(model.state_dict(), model_save_path)
print(f"Modelo guardado en {model_save_path}")

Modelo guardado en nanoGPT_model.pth


In [140]:
# Cargar el estado del modelo guardado
model_load_path = 'nanoGPT_model.pth'
model.load_state_dict(torch.load(model_load_path, map_location=device))
model.eval()  # Configurar el modelo en modo evaluación
print(f"Modelo cargado desde {model_load_path}")

Modelo cargado desde nanoGPT_model.pth


In [141]:
# Prompt inicial (puede ser vacío o un texto)
prompt = "En un lugar de la Mancha"

# Tokenizar el prompt
context_tokens = tokenizer.encode(prompt).ids

# Ajustar el tamaño del contexto si es necesario
if len(context_tokens) > context_size:
    context_tokens = context_tokens[-context_size:]

# Crear el tensor de contexto
context = torch.tensor([context_tokens], dtype=torch.long, device=device)

# Generar texto
max_new_tokens = 200
generated_tokens = model.generate(context, max_new_tokens=max_new_tokens)[0].tolist()

# Decodificar los tokens generados
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(generated_text)

En un lugar de la Manchaza!
¿No están los guerreros Roldán marinas bucles
de futuros cuando ya a todo lo que cuantas?
¿Ahora irán? dime tú, ¿Por qué digo, origen
hallar colmar voz vuestras pétalos perfecta?
¿Dónde hermosa la gente se escorecen,
en vano haciendo virtud aún, coltosas alegría?
¿Y que no de mis aves cualquier quiero­
¿No es pommar? ¿Lo romlanle?
¿Nada es forgados? ¡Oh Ve!
¿Luchosa y a todos? ¿le quisiera? ¿Qué conoces?
¿Acaso quiere una verdad de estatuas no ellas?
 pienses mi madre impetuoso combinviente…
¿Qué usarán ser azul? ¿Desde qué más interpretas?
¿Aquella esclava? ¡ apreta aún decía es vivir.
¿Qué apagan? Gota
me estamos más bien libre recoger mejor
el aprisionado para amargo.
¡Huye es la copa con la conmotuosa tierra!

